In [1]:
import numpy as np
from scipy.special import gamma
import sys
import copy

In [2]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return (idx, array[idx])

## Settings

In [182]:
# casename = 'geometric_IC2'

# # PARAMETERS
# Tmax = 3600*2
# nbin = 32
# dt = 1. #s
# dt_write = 100 #s
# scal = 1 # scalaing factor for grid
# emin = 7.091336e-10 # mg; minimum mass for grid
# rhow = 1e-9 # mg/um^3

# # precip
# r_cutoff = 25
# v_precip = 4/3*np.pi*r_cutoff**3

# nt = int(Tmax/dt)
# nt_write = int(Tmax/dt_write)
# nt_comp_write = int(dt_write/dt)

# printtimes = np.asarray([0, Tmax/4/dt_write, Tmax/dt_write],dtype='int')

# #B = 1500 * 1e-12 #1/sec
# C = np.pi * 1e-12

# # initial condition
# k1 = 1.0
# theta_r = 15.0
# theta = 4/3*np.pi*theta_r**3 #um^3
# N0 = 10.0 # cm^-3

In [202]:
# casename = 'geometric_bimodal'

# # PARAMETERS
# Tmax = 4*3600.0
# nbin = 32
# dt = 5. #s
# dt_write = 100 #s
# scal = 1 # scaling factor for grid
# emin = 7.091336e-10 # mg; minimum mass for grid
# rhow = 1e-9 # mg/um^3

# # precip
# r_cutoff = 25
# v_precip = 4/3*np.pi*r_cutoff**3

# nt = int(Tmax/dt)
# nt_write = int(Tmax/dt_write)
# nt_comp_write = int(dt_write/dt)

# printtimes = np.asarray([0, Tmax/4/dt_write, Tmax/dt_write],dtype='int')

# #B = 1500*1e-12 #1/sec
# #A = 1e-4
# C = np.pi * 1e-12

# # initial condition
# k1 = 4
# theta = 1000 #um^3
# N0 = 10 # cm^-3

# k2 = 2
# theta2 = 200
# N2 = 100

In [22]:
casename = 'golovin_IC1'

# PARAMETERS
Tmax = 3600.0
nbin = 32
dt = 1. #s
dt_write = 120 #s
scal = 1 # scalaing factor for grid
emin = 7.091336e-10 # mg; minimum mass for grid
rhow = 1e-9 # mg/um^3

# precip
r_cutoff = 25
v_precip = 4/3*np.pi*r_cutoff**3

nt = int(Tmax/dt)
nt_write = int(Tmax/dt_write)
nt_comp_write = int(dt_write/dt)

printtimes = np.asarray([0, Tmax/4/dt_write, Tmax/dt_write],dtype='int')

B = 1500 * 1e-12 #1/sec
#C = np.pi * 1e-12

# initial condition
k1 = 3
theta = 1000 #um^3
N0 = 100 # cm^-3

In [204]:
# casename = 'constK_IC2'

# # PARAMETERS
# Tmax = 3600*4
# nbin = 32
# dt = 1. #s
# dt_write = 100 #s
# scal = 1 # scalaing factor for grid
# emin = 7.091336e-10 # mg; minimum mass for grid
# rhow = 1e-9 # mg/um^3

# # precip
# r_cutoff = 25
# v_precip = 4/3*np.pi*r_cutoff**3

# nt = int(Tmax/dt)
# nt_write = int(Tmax/dt_write)
# nt_comp_write = int(dt_write/dt)

# printtimes = np.asarray([0, Tmax/4/dt_write, Tmax/dt_write],dtype='int')

# A = 1e-4 # cm^3 / sec

# # initial condition
# k1 = 1.0
# theta_r = 15.0
# theta = 4/3*np.pi*theta_r**3 #um^3
# N0 = 10.0 # cm^-3

In [205]:
# casename = 'constK'

# # PARAMETERS
# Tmax = 360
# nbin = 32
# dt = 10. #s
# scal = 1 # scalaing factor for grid
# emin = 7.091336e-10 # mg; minimum mass for grid
# rhow = 1e-9 # mg/um^3

# # precip
# v_precip = 1000

# nt = int(Tmax/dt)

# printtimes = np.asarray([0, Tmax/4/dt, Tmax/dt],dtype='int')

# A = 1e-4 # cm^3 / sec

# # initial condition
# k1 = 3
# theta = 100 #um^3
# N0 = 100 # cm^-3

In [206]:
# casename = 'geometric'

# # PARAMETERS
# Tmax = 360
# nbin = 32
# dt = 10. #s
# scal = 1 # scalaing factor for grid
# emin = 7.091336e-10 # mg; minimum mass for grid
# rhow = 1e-9 # mg/um^3

# # precip
# v_precip = 1000

# nt = int(Tmax/dt)

# printtimes = np.asarray([0, Tmax/4/dt, Tmax/dt],dtype='int')

# C = np.pi * 1e-9 # cm^3 / um^4 / sec

# # initial condition
# k1 = 3
# theta = 100 #um^3
# N0 = 100 # cm^-3

## Code

In [23]:
# function for initial condition
def gamma_init(x, N, k, theta):
    try:
        return N*(x/theta)**k/gamma(k)*np.exp(-x/theta)*3*x
    except:
        return 0.0

# functions for kernel
def golovin_kernel(x1,x2,B):
    return B*(x1+x2)

def const_kernel(x1,x2,A):
    return A

def geometric_kernel(x1,x2,C):
    r1 = (x1/np.pi/4*3)**(1/3)
    r2 = (x2/np.pi/4*3)**(1/3)
    a1 = 4 * np.pi * r1**2
    a2 = 4 * np.pi * r2**2
    return C * (r1+r2)**2 * np.abs(a1 - a2)

In [24]:
def courant(vgrid,dlnr):
    nbin = len(vgrid)
    c = np.zeros((nbin,nbin))
    ima = np.zeros((nbin,nbin))
    for i in range(nbin):
        for j in range(i, nbin):
            v0 = vgrid[i] + vgrid[j]
            for k in range(j, nbin):
                if vgrid[k] >= v0 and vgrid[k-1] <= v0:
                    if c[i,j] < 1-1e-8:
                        kk = k-1
                        c[i,j]=np.log(v0/vgrid[kk])/(3*dlnr)
                    else:
                        kk=k
                        c[i,j] = 0.0
                    ima[i,j]=min(nbin-1,kk)
                    break
            c[j,i] = c[i,j]
            ima[j,i] = ima[i,j]
    return (c,ima)


In [25]:
def lin_interp(cck):
    n = len(cck[0])
    ck = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            jm = max(j-1,0)
            im = max(i-1,0)
            jp = min(j+1,n-1)
            ip = min(i+1,n-1)
            ck[i,j] = 0.125*(cck[i,jm]+cck[im,j]+cck[ip,j]+cck[i,jp])+0.5*cck[i,j]
            if i==j:
                ck[i,j]=0.5*ck[i,j]
    return ck

In [26]:
def g_init(x):
    #return gamma_init(x,N0,k1,theta) + gamma_init(x,N2,k2,theta2)
    return gamma_init(x,N0,k1,theta)

# kernel
def kernel(x,y):
    return golovin_kernel(x,y,B)
    #return const_kernel(x,y,A)
    #return geometric_kernel(x,y,C)

In [27]:
# define grid
vmin = emin / rhow
dlnr = np.log(2)/3/scal
gridax = 2**(1/scal)

vgrid = np.zeros(nbin)

vgrid[0] = vmin*0.5*(gridax+1)
for i in range(nbin-1):
    vgrid[i+1] = vgrid[i]*gridax
rgrid = (vgrid/4/np.pi*3)**(1/3)

bin_precip = find_nearest(vgrid,v_precip)[0]+1

# define initial condition
gin = np.zeros(nbin)
for i in range(nbin):
    gin[i] = g_init(vgrid[i])

# define the kernel
cck = np.zeros((nbin,nbin))
for i in range(nbin):
    for j in range(i,nbin):
        cck[i,j] = kernel(vgrid[i],vgrid[j])
        cck[j,i] = cck[i,j]
ck = lin_interp(cck)
ck = ck*dt*dlnr

# compute the courant numbers
(c,ima) = courant(vgrid,dlnr)

In [28]:
def coad(vgrid, gin, ima, ck, c, gmin=1e-8):
    #np.seterr(all='raise')
    g = copy.deepcopy(gin)
    nbin=len(vgrid)
    i0=0
    i1=nbin-1
    # find upper and lower integration limit
    for i in range(nbin):
        i0 = i
        if g[i] >= gmin:
            break
    for i in range(0,nbin-1,-1):
        i1 = i
        if g[i] >= gmin:
            break

    for i in range(i0,i1):
        for j in range(i,i1):
            k = int(ima[i,j])
            kp = k+1
            # try:
            #     v0 = ck[i,j]*g[i]*g[j]
            # except:
            #     v0 = 0.0
            v0 = ck[i,j]*g[i]*g[j]
            v0 = min(v0, g[i]*vgrid[j])
            if j != k:
                v0 = min(v0,g[j]*vgrid[i])
            # try:
            #     gsi=v0/vgrid[j]
            # except:
            #     gsi = 0.0
            gsi=v0/vgrid[j]
            # try:
            #     gsj=v0/vgrid[i]
            # except:
            #     gsj = 0.0
            gsj=v0/vgrid[i]
            gsk=gsi+gsj
            g[i]=max(g[i]-gsi,0)
            #g[i] = g[i] - gsi
            g[j]=max(g[j]-gsj,0)
            #g[j] = g[j] - gsj
            gk=g[k]+gsk
            if gk > gmin:# and g[kp] > gmin:
                try:
                    x1 = np.log(g[kp]/gk+gmin)
                except:
                    print(i,j,k,kp,g[kp],gk)
                flux = gsk/x1*(np.exp(0.5*x1)-np.exp(x1*(0.5-c[i,j])))
                flux = min(min(flux,gk),gsk)
                
                g[k]=gk-flux
                g[kp]+=flux
    #print(g[-1])            
    return g

In [29]:
f = open(casename+'.txt', 'w')
f.close()
f = open(casename+'.txt', 'a')
f.write(str(casename)+'\n')
f.write('Tmax, nbin, dt, scal, emin, rhow, A, k1, theta, N0 \n')
# f.write(str([Tmax, nbin, dt, scal, emin, rhow, B, k1, theta, N0]))
#f.write('Tmax, nbin, dt, scal, emin, rhow, C, k1, theta, N0 \n')
f.write(str([Tmax, nbin, dt, scal, emin, rhow, B, k1, theta, N0]))
f.write('\n')
f.write('\n')
f.write('v_grid = '+np.array2string(vgrid, separator=', ')+'\n')
f.write('r_grid = '+np.array2string((vgrid/4*3/np.pi)**(1/3), separator=', ')+'\n')
f.write('\n')
f.close()


In [30]:
f = open(casename+'.txt', 'a')
bin_mom = np.zeros((3,nt_write+1))
mprecip = np.zeros(nt_write+1)

gprev = gin.copy()
for ij in range(nt_write):
    # calculate the moments, etc.
    bin_mom[0,ij] = np.sum(gprev/vgrid*dlnr)
    bin_mom[1,ij] = np.sum(gprev*dlnr)
    bin_mom[2,ij] = np.sum(gprev*vgrid*dlnr)
    mprecip[ij] = np.sum(gprev[bin_precip:]*dlnr)
    if ij in printtimes:
        print(gprev)
        f.write('gr'+str(ij)+' = '+np.array2string(gprev, separator=', ')+'\n')
    # perform the collision step
    for k in range(nt_comp_write):
        gnext = coad(vgrid, gprev, ima, ck, c)
        gprev = gnext.copy()
bin_mom[0,nt_write] = np.sum(gprev/vgrid*dlnr)
bin_mom[1,nt_write] = np.sum(gprev*dlnr)
bin_mom[2,nt_write] = np.sum(gprev*vgrid*dlnr)
mprecip[nt_write] = np.sum(gprev[bin_precip:]*dlnr)
if ij+1 in printtimes:
    print(gprev)
    f.write('gr'+str(ij+1)+' = '+np.array2string(gprev, separator=', ')+'\n')

mom_times = np.linspace(0.0,Tmax,nt_write+1)
f.write('t_bott = '+np.array2string(mom_times, separator=', ')+'\n')
f.write('M0_bott = '+np.array2string(bin_mom[0,:], separator=', ')+'\n')
f.write('M1_bott = '+np.array2string(bin_mom[1,:], separator=', ')+'\n')
f.write('M2_bott = '+np.array2string(bin_mom[2,:], separator=', ')+'\n')
f.write('mprecip_bin = '+np.array2string(mprecip[:], separator=', ')+'\n')
f.close()

[1.91825611e-007 3.06594679e-006 4.89508996e-005 7.79889052e-004
 1.23724906e-002 1.94619238e-001 3.00969894e+000 4.49860279e+001
 6.28155044e+002 7.65464890e+003 7.10431387e+004 3.82468690e+005
 6.92824760e+005 1.42088340e+005 3.73515426e+002 1.61320203e-004
 1.88074197e-018 1.59768126e-047 7.20596272e-107 9.16169370e-227
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000]
[1.28407317e-07 2.05210249e-06 3.27560499e-05 5.21622716e-04
 8.26735254e-03 1.29797521e-01 1.99961458e+00 2.96610261e+01
 4.07893770e+02 4.82139679e+03 4.21885477e+04 2.09034004e+05
 4.30217174e+05 3.74575869e+05 1.90075870e+05 4.24784992e+04
 3.22377401e+03 7.40206402e+01 6.01773596e-01 2.35307751e-03
 2.78874828e-06 1.99433018e-11 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.0000